Uvozimo potrebne knjižnice za manipulacijo s podatki:

In [2]:
import random
import math

Najprej določimo točke v kvadratu velikosti 1x1. Točke so določena z dvema koordinatama (x,y), kjer x označuje premik po vodoravni stranici, y pa premik po navpični stranici. 

Oglišča kvadrata so (0,0), (0,1), (1,0), (1,1).

In [3]:
def kvadrat_tock(st_tock):
    slovar_tock = {}
    for i in range(st_tock):
        x = random.random()
        y = random.random()
        slovar_tock['tocka_' + str(i)] = (x,y)
    return slovar_tock

tocke = kvadrat_tock(10)
tocke

{'tocka_0': (0.5025343756907436, 0.9138788811742462),
 'tocka_1': (0.05324718022067354, 0.8447043688739839),
 'tocka_2': (0.9405624089842918, 0.4708207661660043),
 'tocka_3': (0.8879025225252024, 0.3412721065519527),
 'tocka_4': (0.12662159603097278, 0.6134934374145571),
 'tocka_5': (0.4621935861724069, 0.7192147793228212),
 'tocka_6': (0.23865264049724522, 0.4733416836430392),
 'tocka_7': (0.7511595918868562, 0.34769998966994453),
 'tocka_8': (0.27378319779064575, 0.497595423256301),
 'tocka_9': (0.19507600695063498, 0.10083803418207371)}

Razdalje med točkami izračunamo s pomočjo Pitagorovega izreka. Glede na položaj točk v prostoru moramo ločiti tri primere.

In [4]:
def razdalje(slovar_tock):
    slovar_razdalj = {}
    #a^2 + b^2 = c^2
    for i in range(len(slovar_tock)):
        seznam_razdalj = []
        for j in range(len(slovar_tock)):
            if (slovar_tock['tocka_'+str(i)][0] > slovar_tock['tocka_'+str(j)][0]) and (slovar_tock['tocka_'+str(i)][1] > slovar_tock['tocka_'+str(j)][1]):
                delta_x = slovar_tock['tocka_'+str(i)][0] - slovar_tock['tocka_'+str(j)][0]
                delta_y = slovar_tock['tocka_'+str(i)][1] - slovar_tock['tocka_'+str(j)][1]

            elif (slovar_tock['tocka_'+str(i)][0] > slovar_tock['tocka_'+str(j)][0]) and (slovar_tock['tocka_'+str(i)][1] < slovar_tock['tocka_'+str(j)][1]):
                delta_x = slovar_tock['tocka_'+str(i)][0] - slovar_tock['tocka_'+str(j)][0]
                delta_y = slovar_tock['tocka_'+str(j)][1] - slovar_tock['tocka_'+str(i)][1]

            else:
                delta_x = slovar_tock['tocka_'+str(j)][0] - slovar_tock['tocka_'+str(i)][0]
                delta_y = slovar_tock['tocka_'+str(j)][1] - slovar_tock['tocka_'+str(i)][1]

            razdalja = sqrt(delta_x^2 + delta_y^2)
            if j == i:
                pass
            else:
                seznam_razdalj.append(('tocka_'+str(j),razdalja))
        slovar_razdalj['tocka_'+str(i)] = seznam_razdalj
    return(slovar_razdalj)

#razdalje(tocke)

Posamezna razdalja nam predstavlja preferenco (manjša kot je razdalja, večja je preferenca). Le te uredimo po velikosti, od najmanjše do največje, za vsako točko:

In [5]:
def preference(slovar_razdalj):
    for i in range(len(slovar_razdalj)):
        sorted_by_second = sorted(slovar_razdalj['tocka_'+str(i)], key=lambda tup: tup[1])
        slovar_razdalj['tocka_'+str(i)] = sorted_by_second
    return(slovar_razdalj)

#preference(razdalje(tocke))

Algoritem je sestavljen iz dveh funkcij:
1) funkcija "Najkrajsa_update"  pregleda razdalje med točkami. Ko najde najmanjšo razdaljo izmed vseh, izbere tisti dve točki, med katerima ta razdalja nastopi in ju poveže v par. Iz slovarja ju nato izbriše, prav tako pa ju izbriše kot preferenco pri ostalih točkah. Na koncu nam vrne par točk, s pripadajočo najmanjšo razdaljo.

In [6]:
def Najkrajsa_update(slovar):
    min_razdalja = sqrt(2)
    par = []
    elementi = []
    while True:
        for key in slovar.keys():
            #print(key)
            if slovar[key][0][1] <= min_razdalja:
                    min_razdalja = slovar[key][0][1]
                    elementi.append([key, min_razdalja])
                    #print('ELEMENTI', elementi)
        # preverimo, če je dovolj elementov za par
        if len(elementi) >= 2:
            # Iz elementov vzamemo samo tocki, ki imata najmanjso razdaljo
            for e in elementi:
                #print(e)
                if e[1] == min_razdalja:
                    par.append(e[0])
            return(par, min_razdalja)


Najkrajsa_update(preference(razdalje(tocke)))

(['tocka_6', 'tocka_8'], 0.0426895764909047)

2) funkcija "Vsi_pari" nam omogoči, da se čez 1. funckijo zapeljemo tolikokrat kolikokrat je potrebno in povežemo vse točke v pare. Na koncu nam vrne seznam vseh parov.

In [7]:
def Vsi_pari(slovar):
    pari = []
    for i in range(len(slovar)/2):
        (par, najkrajsa) = Najkrajsa_update(slovar)
        for e in par:
            # Izbrišemo točki v paru iz slovarja
            if str(e) in slovar:
                del slovar[str(e)]
            # izbrišemo to preferenco še vsem točkam v slovarju
            for elm in slovar.keys():
                #print(elm, slovar[elm])
                for tocka in slovar[elm]:
                    if e in tocka:
                        slovar[elm].remove(tocka)

        pari.append(par)
    return(pari)

#Vsi_pari(preference(razdalje(tocke)))

S funkcijo "vsota_razdalj" izračunamo seštevek vseh razdalj med točkami, ki so skupaj v paru:

In [8]:
def vsota_razdalj(slovar, pari):
    vsota = 0
    for key, item in slovar.items():
        for par in pari:
            if key in par and key == par[0]:
                for element in item:
                    if par[1] in element:
                        vsota += element[1]
                        pari.remove(par)
    return(vsota)

vsota_razdalj(preference(razdalje(tocke)),Vsi_pari(preference(razdalje(tocke))))

1.4532062288565089

Na koncu pa še s pomočjo sledeče funkcije analiziramo, kako se obnaša naša vsota razdalj, ko povečujemo število točk.

In [9]:
def razlicne_vsote(od, do, korak):
    for i in range(od, do, korak):
        yield(vsota_razdalj(preference(razdalje(kvadrat_tock(i))),Vsi_pari(preference(razdalje(kvadrat_tock(i))))))

for i in razlicne_vsote(2, 202, 10):
    print(i)

0.9298530180132284
3.8586681097002313
6.379828362737815
9.277079384121691
11.366927871335461
15.887419754485038
17.065670927597296


17.787810576163942
20.136844737632014


21.621399029196745


24.643114174801017
29.98975573271246


31.693742866254105


...

44.12657475771048


40.89994125313418


43.803784057721


52.76418336866614


52.8579431997296


Jaz tole funkcijo Ujemanje cist ne razumem zakaj je bla tko zapisana. Ampak ostalo je ful dober napisan. Zdej kokr jaz vem oziroma kokr znam jaz naprej rešt je tkole. Jaz bi zdej nadaljeval naprej od preference(razdalje(kvadrat_tock(8))). Ta nama vrze zdej te urejene slovarje po velikosti in to je to kar rabiva. Pol naprej bi pa jaz tko naredu: Zdej morva uredit tiste, ki majo na prvi preferenci isto točko. In to bi nrdila svojo funkcijo. Se pravi, če imata dva (v našem primeru točka 4 in 6) na prvem mesti isto točko (v našem primeru imata točko 3), potem pogledamo k tej isti točki (točka 3), katera od teh dveh točk je na njenem seznamu prej (vidimo, da je to točka 4). Zato na seznamu točke 3 zbrišemo vse točke za točko 4. In za te točke, ki smo jih zbrisali, potem na njihovih seznamih zbrišemo točko 3. Potem avtomatsko dobimo novo prvo preferenčno točka na točki 6. In pol to funkcijo zapremo. Tko dobimo novo urejene sezname. 

ežko razložim ta del: Zdej pa ubistvu samo še pogledamo pri vsaki točki kaj je njena preferenčna točka in na seznamu te preferenčne funkcije  izbrišemo vse točke za tisto točko, ki ima to točko za preferenčno točko. in za te izbrisane točke še izbrišemo točke na njenem seznamu.


In [9]:
def faza_1(slovar_preferenc):
    reduciran_slovar = {}
    for i in range(len(slovar_preferenc)):
        kljuc_i = list(slovar.keys())[i]
        for j in range(len(slovar_preferenc)):
            kljuc_j = list(slovar.keys())[j]
            if i!=j:
                if list(slovar.values())[i][0][0] = list(slovar.values())[j][0][0]:
                    ujemajoca_tocka = list(slovar.values())[i][0][0]
                    seznam_ujemajoce_tocke = print(slovar['tocka_'+str(ujemajoca_tocka
                    

faza_1(preference(razdalje(kvadrat_tock(8))))

SyntaxError: invalid syntax (<ipython-input-9-3cf796c9e6f6>, line 8)